In [195]:
# Import packages
from logging.config import valid_ident
from pydoc import resolve
import ee
import pandas as pd
import numpy as np
import time
from geetools import batch
from tqdm import tqdm
from argparse import ArgumentParser
import requests
import math
import logging
import multiprocessing
import requests
from retry import retry
import multiprocessing
import os
import csv
from functools import partial
import folium

In [196]:
# Personal credentials
service_account = "sentinel2@uhi-causal-model.iam.gserviceaccount.com"
json_key = "uhi-causal-model-a7e891b460a5.json"

In [197]:
# Initialize the library
ee.Initialize(ee.ServiceAccountCredentials(service_account, json_key), opt_url='https://earthengine-highvolume.googleapis.com')

In [198]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [199]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data © Google Earth Engine',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [200]:
# Look for bounding box at a specific location
def bounding_box(lat, lon, size, res):

  earthRadius = 6371000
  angularDist = math.degrees(0.5 * ((size * res) / earthRadius))
  osLat = angularDist
  osLon = angularDist
  xMin = lon - osLon
  xMax = lon + osLon
  yMin = lat - osLat
  yMax = lat + osLat
  return xMin, xMax, yMin, yMax

In [201]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [202]:
# Import dataset
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")

# Define date
startDate = '2019-07-29'
endDate = '2019-07-30'
lat,lon = 42.325347,-71.120802
height = 512
width = 512
res = 60

# Visualizer
RGB_VIZ = {'min':0, 'max':255, 'bands':['TCI_R', 'TCI_G', 'TCI_B']}


In [203]:
# Display true color image
xMin, xMax, yMin, yMax = bounding_box(lat, lon, height, res)
geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])

filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select(['TCI_R', 'TCI_G', 'TCI_B'])
image = filtered.median().clip(geometry)
map = folium.Map(location=[lat, lon], zoom_start=11)
map.add_ee_layer(image, RGB_VIZ, 'rgb')
display(map)

In [204]:
filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select(['B8', 'B4'])
image = filtered.median().clip(geometry)
ndvi = image.normalizedDifference(['B8', 'B4'])
map = folium.Map(location=[lat, lon], zoom_start=11)
ndvi_params = {'min': -1, 'max': 1, 'palette': ['#640000','#ff0000','#ffff00','#00c800','#006400']}
map.add_ee_layer(ndvi, ndvi_params, 'NDVI')
display(map)


In [205]:
filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select(['SCL'])
image = filtered.median().clip(geometry)
map = folium.Map(location=[lat, lon], zoom_start=11)
scl_params = {'min': 1, 'max': 11, 'bands': ['SCL'], 'palette':['#ff0004','#868686','#774b0a','#10d22c','#ffff52','#0000ff','#818181','#c0c0c0','#f1f1f1','#bac5eb','#52fff9']}
map.add_ee_layer(image, scl_params, 'SCL')
display(map)

In [192]:

height = 450
width = 600
description = f"boston_rgb"
crs='EPSG:3857'
res = 60 # Redefine resolution to 
xMin, xMax, yMin, yMax = bounding_box(lat, lon, height, res)
geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])
filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select('B8', 'B4', 'B3', 'B2')
# cloud_pct = 10
# filtered = filtered.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_pct))
image = filtered.median().clip(geometry)
bands = ['B4', 'B3', 'B2']
_min = 0
_max = 4500
band_names = image.bandNames()
bands_list = band_names.getInfo()
if all(band in bands_list for band in bands):
    image_vis = image.visualize(bands=bands, min=_min, max=_max)
    try:
        url = image_vis.getDownloadUrl({
            'description': description,
            'region': geometry,
            'fileNamePrefix': description,
            'crs': crs,
            'fileFormat': 'GEO_TIFF',
            'region': geometry,
            'format': 'GEO_TIFF',
            'dimensions': [height, width]
        })
        # download image given URL
        response = requests.get(url)
        if response.status_code != 200:
            raise response.raise_for_status()
        with open(os.path.join('data/', f'{description}.tif'), 'wb') as fd:
            fd.write(response.content)
        logging.info(f'Done: {description}')

    except Exception as e:
        logging.exception(e)

else:
    logging.info(f'Image at {(lat, lon)} has bands: {bands_list}')
    pass

In [168]:
RGB = ['B4', 'B3', 'B2']
RGB_VIZ = {'min':0, 'max': 4500, 'bands':RGB, 'opacity': 1}
res = 60
crs = 'EPSG:3857'
crs2 = 'EPSG:4326'
fileFormat = 'GeoTIFF'
cloud_percent = 20 
cloud_filter = 10

description = f'Boston'
xMin, xMax, yMin, yMax = bounding_box(lat, lon, height, res)
geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])


filtered = sentinel.filterDate(startDate, endDate).filterBounds(geometry).select(RGB)

image = filtered.median().clip(geometry)

# imgVis = image.visualize(bands = RGB, min = 0, max = 0.5)
# OPTIONAL: just to view the map, to comment out when exporting more than 1 image
map = folium.Map(location=[lat, lon], zoom_start=11)
map.add_ee_layer(image, RGB_VIZ, 'rgb')
display(map)